In [1]:
import pandas as pd
import numpy as np
import math
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('names')
import re

[nltk_data] Downloading package wordnet to C:\Users\Caique de
[nltk_data]     Camargo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Caique de
[nltk_data]     Camargo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Caique de
[nltk_data]     Camargo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to C:\Users\Caique de
[nltk_data]     Camargo\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df=df[["does_this_tweet_contain_hate_speech", "tweet_text"]]

In [4]:
df.rename(columns={ 'does_this_tweet_contain_hate_speech':'Label',
                    'tweet_text':'Tweet'}, 
                 inplace=True)

In [5]:
df['Label'] = df['Label'].astype('category')
df['Label'] = df['Label'].cat.rename_categories(['hate','inoffensive','offensive'])

In [6]:
def regex(x):
    x=' '.join(re.sub("@([A-Za-z0-9_À-ÖØ-öø-ÿ.]+)|:|RT|#([A-Za-z0-9_À-ÖØ-öø-ÿ.]+)|\"|http([A-Za-z0-9_À-ÖØ-öø-ÿ./:]+)|[À-ÖØ-öø-ÿ�㢉۝ʉӉ~0-9ʁ().!?,]|.co([A-Za-z0-9_À-ÖØ-öø-ÿ./]+)|[_&;//]+"," ",x).split())
    return x

In [7]:
df['Tweet'] = df['Tweet'].apply(regex)

In [8]:
df.head(10)

,Label,Tweet
0,offensive,Warning penny boards will make you a faggot
1,hate,Fuck dykes
2,hate,at least i dont look like jefree starr faggot
3,hate,Is a fag jackie jealous Neeeee
4,offensive,You heard me bitch but any way I'm back th tex...
5,hate,your a dirty terrorist and your religion is a ...
6,hate,looking like faggots
7,hate,Well I thought you knew actually Man why y'all...
8,offensive,I know It was a joke faggot
9,offensive,I'm tired of people saying I look like my brot...


In [9]:
hate = df[df.Label == 'hate'].values
offensive = df[df.Label == 'offensive'].values
inoffensive = df[df.Label == 'inoffensive'].values
train = np.concatenate((hate[:100],offensive[:100],inoffensive[:100]), axis=0)
stopWords = set(stopwords.words('english'))

In [10]:
all_words = []
for passage in train:
    for word in word_tokenize(passage[1]):
        if word not in stopWords:
            all_words.append(word.lower())
#all_words = set(word.lower() for passage in train for word in word_tokenize(passage[1]))

dictionary = [({word: (word in word_tokenize(x[1])) for word in all_words}, x[0]) for x in train]

In [11]:
classifier_NB = nltk.NaiveBayesClassifier.train(dictionary)
classifier_NB.show_most_informative_features()

Most Informative Features
                 bastard = True           offens : hate   =     25.0 : 1.0
                   alike = True           inoffe : offens =     13.4 : 1.0
                  faggot = True             hate : offens =      8.7 : 1.0
                    look = True           inoffe : hate   =      8.3 : 1.0
                    like = True             hate : inoffe =      5.8 : 1.0
                 bitches = True           offens : hate   =      5.7 : 1.0
                     not = True           offens : hate   =      5.7 : 1.0
                   bitch = True           offens : hate   =      5.0 : 1.0
                   great = True           inoffe : offens =      5.0 : 1.0
                     now = True           offens : hate   =      4.3 : 1.0


In [12]:
test = ['black', 'plus', 'female']
for t in test:
    print(t)
    sent_features = {word: (word == t) for word in all_words}
    classify = classifier_NB.classify(sent_features)
    acc = [({word: (word == t)}, classify) for word in all_words]
    print(classifier_NB.classify(sent_features))
    print(nltk.classify.accuracy(classifier_NB, acc))
    print()

black
hate
0.14417942328230687

plus
inoffensive
0.35599857600569595

female
hate
0.14560341758632966



In [13]:
classifier_SVM = nltk.classify.SklearnClassifier(LinearSVC())
classifier_SVM.train(dictionary)
for t in test:
    print(t)
    sent_features = {word: (word == t) for word in all_words}
    classify = classifier_SVM.classify(sent_features)
    acc = [({word: (word == t)}, classify) for word in all_words]
    print(classifier_SVM.classify(sent_features))
    print(nltk.classify.accuracy(classifier_SVM, acc))
    print()

black
inoffensive
1.0

plus
inoffensive
1.0

female
inoffensive
1.0

